#ZIP EXTRACTION / 
#DOWNLOAD -> BRONZE
---

In [0]:
import os, zipfile, shutil

DOWNLOADS = "abfss://citibike@databricksjm.dfs.core.windows.net/downloads"
BRONZE = "abfss://citibike@databricksjm.dfs.core.windows.net/bronze"
LOCAL_TMP = "/local_disk0/tmp/citibike"
EXTRACTED = f"{LOCAL_TMP}/extracted"
adls_files = dbutils.fs.ls(DOWNLOADS)

# -przechodzi do external data (ADLS) -> folderu downloads
# -iteruje po plikach (ADLS)

skipped = 0
processed = 0

for adls_file in adls_files:

    # nazwa pliku adls
    adls_filename = adls_file.name

    # pobranie roku z nazwy pliku
    year = adls_filename[:4]
    
    print(f"Now processing: {adls_filename}")

    # tymczasowe..do zmiany
    if year not in ("2024", "2025"):
        try:
            dbutils.fs.ls(f"{BRONZE}/{year}")
            print(f"{year} Folder already exists.\nSkipping...")
            skipped += 1
            continue
        except:
            print(f"{year} folder doesnt exist.\nContinuing...")

    # tworzenie folderu lokalnego
    os.makedirs(LOCAL_TMP, exist_ok=True)
    print(f"Created local folder: {LOCAL_TMP}")

    # PRZYGOTOWANIE DO KOPII (ADLS -> LOCAL)
    # zrodlo
    source_path = f"{DOWNLOADS}/{adls_filename}"
    # cel
    destination_path = f"file:{LOCAL_TMP}/{adls_filename}"
    # przepisanie pliku z ADLS do lokalnego folderu}"
    dbutils.fs.cp(source_path, destination_path)
    print(f"File copied from ADLS to local: {adls_filename}")

    # iteracja po local (citibike)
    print("Iterating through local...")
    for local_file in os.listdir(LOCAL_TMP):
        # sprawdzanie czy zip
        if local_file.lower().endswith(".zip"):
            print(".zip file found")
            # tworzenie sciezki zipa
            zip_path = os.path.join(LOCAL_TMP, local_file)
            # tworzenie folderu do wypakowywania
            if year in ("2024", "2025"):
                os.makedirs(f"{LOCAL_TMP}/{year}", exist_ok=True)
                with zipfile.ZipFile(zip_path, "r") as z:
                    z.extractall(f"{LOCAL_TMP}/{year}")
                dbutils.fs.cp(f"file:{LOCAL_TMP}/{year}", f"{BRONZE}/{year}", recurse=True)
                shutil.rmtree(LOCAL_TMP)
                break
            os.makedirs(EXTRACTED, exist_ok=True)
            # wypakowanie do extracted
            print(f"Extracting zip file to {EXTRACTED}...")
            with zipfile.ZipFile(zip_path, "r") as z:
                z.extractall(EXTRACTED)
            # iteracja po extracted
            print("Iterating through extracted files...")
            for extracted_file in os.listdir(EXTRACTED):
                # sprawdzanie czy to folder o tej samej nazwie jak zip
                if extracted_file == adls_filename[:-4]:
                    # tworzenie folderu roku
                    os.makedirs(f"{EXTRACTED}/{extracted_file}/{year}", exist_ok=True)
                    # iteracja po plikach w folderze (tym rozpakowanym)
                    for file_in_extracted_file in os.listdir(f"{EXTRACTED}/{extracted_file}"):
                        # sprawdzanie czy to folder typu "1_" lub "12_" itp.
                        if (file_in_extracted_file[1] == '_') or (file_in_extracted_file[2] == '_'):
                            print("Processing old type")
                            # jesli tak to iteracja po tym folderze
                            for file_in_extracted_file_2 in os.listdir(f"{EXTRACTED}/{extracted_file}/{file_in_extracted_file}"):
                                # przepisanie zawartosci do folderu z rokiem
                                # zrodlo
                                source_path = f"{EXTRACTED}/{extracted_file}/{file_in_extracted_file}/{file_in_extracted_file_2}"
                                # cel
                                destination_path = f"{EXTRACTED}/{extracted_file}/{year}"
                                # transfer
                                shutil.move(source_path, destination_path)
                            # jesli to nie folder typu "1_" lub "12_" to sprawdzanie czy jest zipem
                        elif file_in_extracted_file.endswith(".zip"):
                            print("Processing new type")
                            # jesli zip to wypakowanie do folderu roku
                            with zipfile.ZipFile(f"{EXTRACTED}/{extracted_file}/{file_in_extracted_file}", "r") as z:
                                z.extractall(f"{EXTRACTED}/{extracted_file}/{year}")
                        else: continue
                    # przeniesienie do ADLS
                    print(f"Moving extracted files to ADLS: {year}")
                    local = f"{EXTRACTED}/{extracted_file}/{year}"
                    dbutils.fs.cp(f"file:{local}", f"{BRONZE}/{year}", recurse=True)
                    # usuniecie folderu lokalnego citibike
                    shutil.rmtree(LOCAL_TMP)
                    print(f"Removed local folder: {LOCAL_TMP}")
                    print("############### DONE ############### ")
                    processed += 1
            break
print(f"Processed: {processed} | Skipped: {skipped}")